## Fetching Raster Data from GDI using SDK

Below script runs the command line to fetch vector data using GDI SDK
1. The client credentials (id and secret) are made avialable for download at the registering as user of GDI platform. In case credentials are lost, go to your profile, reset the credentials and download for further use.
2. In GDI, rasters are onbaorded as STAC(spatio temporal asset catalog). Hence data can be fetched as collection/ item level. 
     - If user specifies collection_id/dataset_id, all the items with its raster assets will be downloaded
     - If user specified collection_id and item_id, that particular item with easter assets from the collection will be downloaded.
3. For each data of interest, collection_id can be copied from catalogue page, available as dataset_id. In case of item_id, it is the label which can be copied from STAC browser.
4. To save the fetched data, user will specify a folder name, so that all the rasters even at item level will be saved to the folder.


In [ ]:
'''
Install gdi python SDK using below pip statement
pip install git+https://github.com/datakaveri/gdi-python-sdk.git
-------------------------------------------------------------------------------
Command line:
gdi get-raster-data --client-id <client_id> --client-secret <client_secret> --role consumer --collection-ids <collection_id> --config-path config.json --dir-path <output_folder> --item-id <item_id>

where --item-id is optional.
'''

### Function definition to run the command line to fetch the data

In [ ]:
import os
import subprocess

def fetch_raster(client_id, client_secret, collection_id, folder_name, item_id=None):
    """
    Downloads raster data from the GDI SDK for a given collection or item.
    """

    # Define the output directory path
    base_dir = "./Data"
    output_dir = os.path.join(base_dir, folder_name)
    os.makedirs(output_dir, exist_ok=True)

    # Construct the GDI SDK command
    cmd = [
        "gdi", "get-raster-data",
        "--client-id", client_id,
        "--client-secret", client_secret,
        "--role", "consumer",
        "--collection-ids", collection_id,
        "--store-artifact", "local",
        "--dir-path", output_dir
    ]

    # Add item-id if provided
    if item_id:
        cmd.extend(["--item-id", item_id])

    # Print command info for debugging
    print(f"[INFO] Executing command:\n{' '.join(cmd)}\n")

    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("[SUCCESS] Raster data saved to folder:", output_dir)

        if result.stdout.strip():
            print("\n--- STDOUT ---\n", result.stdout)

    except subprocess.CalledProcessError as e:
        print("[ERROR] Failed to fetch raster data.")
        print(e.stderr)


### a. Run below code to fetch all rasters in a stac collection

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import os

# Create widgets for entire collection download
client_id_widget = widgets.Text(description="Client ID:", layout=widgets.Layout(width='90%'))
client_secret_widget = widgets.Password(description="Client Secret:", layout=widgets.Layout(width='90%'))
collection_id_widget = widgets.Text(description="Collection ID:", layout=widgets.Layout(width='90%'))
folder_name_widget = widgets.Text(description="Folder Name:", layout=widgets.Layout(width='90%'))
submit_button_all = widgets.Button(description="Fetch Collection", button_style="success")
output_all = widgets.Output()

# Debounce timestamp
last_click_time_all = 0

def on_submit_all(b):
    global last_click_time_all

    current_time = time.time()
    if current_time - last_click_time_all < 0.5:
        return
    last_click_time_all = current_time

    with output_all:
        clear_output()
        print("[INFO] Starting collection download...")

    client_id = client_id_widget.value.strip()
    client_secret = client_secret_widget.value.strip()
    collection_id = collection_id_widget.value.strip()
    folder_name = folder_name_widget.value.strip()

    try:
        fetch_raster(client_id, client_secret, collection_id, folder_name)

        output_dir = os.path.join("./Data", folder_name)

        # Gather all files in the output directory
        files_list = []
        for root, dirs, files in os.walk(output_dir):
            for f in files:
                rel_dir = os.path.relpath(root, output_dir)
                rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
                files_list.append(rel_file)

        with output_all:
            print("[DONE] Collection download complete.")
            print(f"\nFiles saved to folder: {output_dir}")
            if files_list:
                print("\nDownloaded files:")
                for f in files_list:
                    print(f"- {f}")
            else:
                print("No files found in the folder.")

    except Exception as e:
        with output_all:
            print("[ERROR]", e)

# Ensure no duplicate handlers
submit_button_all._click_handlers.callbacks = []
submit_button_all.on_click(on_submit_all)

# Display widgets for entire collection
display(
    client_id_widget,
    client_secret_widget,
    collection_id_widget,
    folder_name_widget,
    submit_button_all,
    output_all
)


Text(value='', description='Client ID:', layout=Layout(width='90%'))

Password(description='Client Secret:', layout=Layout(width='90%'))

Text(value='', description='Collection ID:', layout=Layout(width='90%'))

Text(value='', description='Folder Name:', layout=Layout(width='90%'))

Button(button_style='success', description='Fetch Collection', style=ButtonStyle())

Output()

### b. Run below code to fetch all rasters in a stac item of a collection

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import os

# Create widgets for single item download
client_id_widget_item = widgets.Text(description="Client ID:", layout=widgets.Layout(width='90%'))
client_secret_widget_item = widgets.Password(description="Client Secret:", layout=widgets.Layout(width='90%'))
collection_id_widget_item = widgets.Text(description="Collection ID:", layout=widgets.Layout(width='90%'))
item_id_widget = widgets.Text(description="Item ID:", layout=widgets.Layout(width='90%'))
folder_name_widget_item = widgets.Text(description="Folder Name:", layout=widgets.Layout(width='90%'))
submit_button_item = widgets.Button(description="Fetch Item", button_style="success")
output_item = widgets.Output()

# Debounce timestamp
last_click_time_item = 0

def on_submit_item(b):
    global last_click_time_item

    current_time = time.time()
    if current_time - last_click_time_item < 0.5:
        return
    last_click_time_item = current_time

    with output_item:
        clear_output()
        print("[INFO] Starting item download...")

    client_id = client_id_widget_item.value.strip()
    client_secret = client_secret_widget_item.value.strip()
    collection_id = collection_id_widget_item.value.strip()
    item_id = item_id_widget.value.strip()
    folder_name = folder_name_widget_item.value.strip()

    try:
        fetch_raster(client_id, client_secret, collection_id, folder_name, item_id=item_id)

        output_dir = os.path.join("./Data", folder_name)

        # Gather all files in the output directory
        files_list = []
        for root, dirs, files in os.walk(output_dir):
            for f in files:
                rel_dir = os.path.relpath(root, output_dir)
                rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
                files_list.append(rel_file)

        with output_item:
            print("[DONE] Item download complete.")
            print(f"\nFiles saved to folder: {output_dir}")
            if files_list:
                print("\nDownloaded files:")
                for f in files_list:
                    print(f"- {f}")
            else:
                print("No files found in the folder.")

    except Exception as e:
        with output_item:
            print("[ERROR]", e)

# Ensure no duplicate handlers
submit_button_item._click_handlers.callbacks = []
submit_button_item.on_click(on_submit_item)

# Display widgets for single item download
display(
    client_id_widget_item,
    client_secret_widget_item,
    collection_id_widget_item,
    item_id_widget,
    folder_name_widget_item,
    submit_button_item,
    output_item
)


Text(value='', description='Client ID:', layout=Layout(width='90%'))

Password(description='Client Secret:', layout=Layout(width='90%'))

Text(value='', description='Collection ID:', layout=Layout(width='90%'))

Text(value='', description='Item ID:', layout=Layout(width='90%'))

Text(value='', description='Folder Name:', layout=Layout(width='90%'))

Button(button_style='success', description='Fetch Item', style=ButtonStyle())

Output()